In [52]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

from datetime import datetime as dt

In [53]:
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/'

excluded = ["Guam", "District of Columbia", "Grand Princess",
            "Diamond Princess","Northern Mariana Islands",
            "Puerto Rico", "Recovered", "American Samoa", "Virgin Islands"]

In [54]:
dates = pd.date_range(start='04-12-2020', end='07-26-2022').strftime('%m-%d-%Y')
for date in dates[0:5]:
    print(date)
print()
print(f'Total Days: {len(dates)}')

04-12-2020
04-13-2020
04-14-2020
04-15-2020
04-16-2020

Total Days: 836


In [55]:
def generateSmallTables(dates = dates):
    for date in dates:
        url = BASE_URL + date + ".csv"
        print(url)
        df = pd.read_csv(url).fillna(0).
        df.to_csv("data/SmallTables/" + date + ".csv", index=False)

#generateSmallTables()

In [60]:
def editSmallTables(dates = dates):
    for date in dates:
        path = "data/SmallTables/" + date + ".csv"
        print(path)
        df = pd.read_csv(path).fillna(0)
        df['Last_Update'] = date
        df.to_csv("data/SmallTables/" + date + ".csv", index=False)

editSmallTables()

data/SmallTables/04-12-2020.csv
data/SmallTables/04-13-2020.csv
data/SmallTables/04-14-2020.csv
data/SmallTables/04-15-2020.csv
data/SmallTables/04-16-2020.csv
data/SmallTables/04-17-2020.csv
data/SmallTables/04-18-2020.csv
data/SmallTables/04-19-2020.csv
data/SmallTables/04-20-2020.csv
data/SmallTables/04-21-2020.csv
data/SmallTables/04-22-2020.csv
data/SmallTables/04-23-2020.csv
data/SmallTables/04-24-2020.csv
data/SmallTables/04-25-2020.csv
data/SmallTables/04-26-2020.csv
data/SmallTables/04-27-2020.csv
data/SmallTables/04-28-2020.csv
data/SmallTables/04-29-2020.csv
data/SmallTables/04-30-2020.csv
data/SmallTables/05-01-2020.csv
data/SmallTables/05-02-2020.csv
data/SmallTables/05-03-2020.csv
data/SmallTables/05-04-2020.csv
data/SmallTables/05-05-2020.csv
data/SmallTables/05-06-2020.csv
data/SmallTables/05-07-2020.csv
data/SmallTables/05-08-2020.csv
data/SmallTables/05-09-2020.csv
data/SmallTables/05-10-2020.csv
data/SmallTables/05-11-2020.csv
data/SmallTables/05-12-2020.csv
data/Sma

data/SmallTables/01-02-2021.csv
data/SmallTables/01-03-2021.csv
data/SmallTables/01-04-2021.csv
data/SmallTables/01-05-2021.csv
data/SmallTables/01-06-2021.csv
data/SmallTables/01-07-2021.csv
data/SmallTables/01-08-2021.csv
data/SmallTables/01-09-2021.csv
data/SmallTables/01-10-2021.csv
data/SmallTables/01-11-2021.csv
data/SmallTables/01-12-2021.csv
data/SmallTables/01-13-2021.csv
data/SmallTables/01-14-2021.csv
data/SmallTables/01-15-2021.csv
data/SmallTables/01-16-2021.csv
data/SmallTables/01-17-2021.csv
data/SmallTables/01-18-2021.csv
data/SmallTables/01-19-2021.csv
data/SmallTables/01-20-2021.csv
data/SmallTables/01-21-2021.csv
data/SmallTables/01-22-2021.csv
data/SmallTables/01-23-2021.csv
data/SmallTables/01-24-2021.csv
data/SmallTables/01-25-2021.csv
data/SmallTables/01-26-2021.csv
data/SmallTables/01-27-2021.csv
data/SmallTables/01-28-2021.csv
data/SmallTables/01-29-2021.csv
data/SmallTables/01-30-2021.csv
data/SmallTables/01-31-2021.csv
data/SmallTables/02-01-2021.csv
data/Sma

data/SmallTables/09-23-2021.csv
data/SmallTables/09-24-2021.csv
data/SmallTables/09-25-2021.csv
data/SmallTables/09-26-2021.csv
data/SmallTables/09-27-2021.csv
data/SmallTables/09-28-2021.csv
data/SmallTables/09-29-2021.csv
data/SmallTables/09-30-2021.csv
data/SmallTables/10-01-2021.csv
data/SmallTables/10-02-2021.csv
data/SmallTables/10-03-2021.csv
data/SmallTables/10-04-2021.csv
data/SmallTables/10-05-2021.csv
data/SmallTables/10-06-2021.csv
data/SmallTables/10-07-2021.csv
data/SmallTables/10-08-2021.csv
data/SmallTables/10-09-2021.csv
data/SmallTables/10-10-2021.csv
data/SmallTables/10-11-2021.csv
data/SmallTables/10-12-2021.csv
data/SmallTables/10-13-2021.csv
data/SmallTables/10-14-2021.csv
data/SmallTables/10-15-2021.csv
data/SmallTables/10-16-2021.csv
data/SmallTables/10-17-2021.csv
data/SmallTables/10-18-2021.csv
data/SmallTables/10-19-2021.csv
data/SmallTables/10-20-2021.csv
data/SmallTables/10-21-2021.csv
data/SmallTables/10-22-2021.csv
data/SmallTables/10-23-2021.csv
data/Sma

data/SmallTables/07-15-2022.csv
data/SmallTables/07-16-2022.csv
data/SmallTables/07-17-2022.csv
data/SmallTables/07-18-2022.csv
data/SmallTables/07-19-2022.csv
data/SmallTables/07-20-2022.csv
data/SmallTables/07-21-2022.csv
data/SmallTables/07-22-2022.csv
data/SmallTables/07-23-2022.csv
data/SmallTables/07-24-2022.csv
data/SmallTables/07-25-2022.csv
data/SmallTables/07-26-2022.csv


In [56]:
def moveSmallTables(dates = dates):
    for date in dates:
        path = "data/" + date + ".csv"
        print(path)
        pd.read_csv(path).fillna(0).to_csv("data/SmallTables/" + date + ".csv", index=False)

#moveSmallTables()

In [63]:
def generateBigTable(dates = dates):
    allData = pd.DataFrame()
    for date in dates:
        path = "data/SmallTables/" + date + ".csv"
        print(path)
        allData = pd.concat([allData, pd.read_csv(path).fillna(0)])
    # There is probably a better way to clean this but it isn't very clear
    allData = allData[allData.Province_State != "Guam"]
    allData = allData[allData.Province_State != "District of Columbia"]
    allData = allData[allData.Province_State != "Grand Princess"]
    allData = allData[allData.Province_State != "Diamond Princess"]
    allData = allData[allData.Province_State != "Northern Mariana Islands"]
    allData = allData[allData.Province_State != "Puerto Rico"]
    allData = allData[allData.Province_State != "Recovered"]
    allData = allData[allData.Province_State != "American Samoa"]
    allData = allData[allData.Province_State != "Virgin Islands"]
    
    #allData = allData.drop(allData.columns[[0]],axis = 1)
    allData.to_csv("data/bigTable.csv", index=False)

generateBigTable()

data/SmallTables/04-12-2020.csv
data/SmallTables/04-13-2020.csv
data/SmallTables/04-14-2020.csv
data/SmallTables/04-15-2020.csv
data/SmallTables/04-16-2020.csv
data/SmallTables/04-17-2020.csv
data/SmallTables/04-18-2020.csv
data/SmallTables/04-19-2020.csv
data/SmallTables/04-20-2020.csv
data/SmallTables/04-21-2020.csv
data/SmallTables/04-22-2020.csv
data/SmallTables/04-23-2020.csv
data/SmallTables/04-24-2020.csv
data/SmallTables/04-25-2020.csv
data/SmallTables/04-26-2020.csv
data/SmallTables/04-27-2020.csv
data/SmallTables/04-28-2020.csv
data/SmallTables/04-29-2020.csv
data/SmallTables/04-30-2020.csv
data/SmallTables/05-01-2020.csv
data/SmallTables/05-02-2020.csv
data/SmallTables/05-03-2020.csv
data/SmallTables/05-04-2020.csv
data/SmallTables/05-05-2020.csv
data/SmallTables/05-06-2020.csv
data/SmallTables/05-07-2020.csv
data/SmallTables/05-08-2020.csv
data/SmallTables/05-09-2020.csv
data/SmallTables/05-10-2020.csv
data/SmallTables/05-11-2020.csv
data/SmallTables/05-12-2020.csv
data/Sma

data/SmallTables/01-21-2021.csv
data/SmallTables/01-22-2021.csv
data/SmallTables/01-23-2021.csv
data/SmallTables/01-24-2021.csv
data/SmallTables/01-25-2021.csv
data/SmallTables/01-26-2021.csv
data/SmallTables/01-27-2021.csv
data/SmallTables/01-28-2021.csv
data/SmallTables/01-29-2021.csv
data/SmallTables/01-30-2021.csv
data/SmallTables/01-31-2021.csv
data/SmallTables/02-01-2021.csv
data/SmallTables/02-02-2021.csv
data/SmallTables/02-03-2021.csv
data/SmallTables/02-04-2021.csv
data/SmallTables/02-05-2021.csv
data/SmallTables/02-06-2021.csv
data/SmallTables/02-07-2021.csv
data/SmallTables/02-08-2021.csv
data/SmallTables/02-09-2021.csv
data/SmallTables/02-10-2021.csv
data/SmallTables/02-11-2021.csv
data/SmallTables/02-12-2021.csv
data/SmallTables/02-13-2021.csv
data/SmallTables/02-14-2021.csv
data/SmallTables/02-15-2021.csv
data/SmallTables/02-16-2021.csv
data/SmallTables/02-17-2021.csv
data/SmallTables/02-18-2021.csv
data/SmallTables/02-19-2021.csv
data/SmallTables/02-20-2021.csv
data/Sma

data/SmallTables/10-27-2021.csv
data/SmallTables/10-28-2021.csv
data/SmallTables/10-29-2021.csv
data/SmallTables/10-30-2021.csv
data/SmallTables/10-31-2021.csv
data/SmallTables/11-01-2021.csv
data/SmallTables/11-02-2021.csv
data/SmallTables/11-03-2021.csv
data/SmallTables/11-04-2021.csv
data/SmallTables/11-05-2021.csv
data/SmallTables/11-06-2021.csv
data/SmallTables/11-07-2021.csv
data/SmallTables/11-08-2021.csv
data/SmallTables/11-09-2021.csv
data/SmallTables/11-10-2021.csv
data/SmallTables/11-11-2021.csv
data/SmallTables/11-12-2021.csv
data/SmallTables/11-13-2021.csv
data/SmallTables/11-14-2021.csv
data/SmallTables/11-15-2021.csv
data/SmallTables/11-16-2021.csv
data/SmallTables/11-17-2021.csv
data/SmallTables/11-18-2021.csv
data/SmallTables/11-19-2021.csv
data/SmallTables/11-20-2021.csv
data/SmallTables/11-21-2021.csv
data/SmallTables/11-22-2021.csv
data/SmallTables/11-23-2021.csv
data/SmallTables/11-24-2021.csv
data/SmallTables/11-25-2021.csv
data/SmallTables/11-26-2021.csv
data/Sma

data/SmallTables/07-26-2022.csv


In [64]:
testView = pd.read_csv("data/bigTable.csv")
print(testView.shape)
testView.head(10)

(41800, 21)


,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,...,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,Date,People_Tested,Mortality_Rate
0,Alabama,US,04-12-2020,32.318,-86.902,3667,93,0.000,0.000,1.000,...,0.000,437.000,0.000,84000001.000,USA,1265.076,12.265,2020-04-12,62029.000,2.536
1,Alaska,US,04-12-2020,61.371,-152.404,272,8,66.000,198.000,2.000,...,0.000,31.000,0.000,84000002.000,USA,1344.712,11.397,2020-04-12,8038.000,2.941
2,Arizona,US,04-12-2020,33.730,-111.431,3542,115,0.000,0.000,4.000,...,0.000,0.000,0.000,84000004.000,USA,578.522,0.000,2020-04-12,52289.000,3.247
3,Arkansas,US,04-12-2020,34.970,-92.373,1280,27,367.000,886.000,5.000,...,0.000,130.000,0.000,84000005.000,USA,761.753,10.156,2020-04-12,19722.000,2.109
4,California,US,04-12-2020,36.116,-119.682,22201,632,0.000,0.000,6.000,...,0.000,5234.000,0.000,84000006.000,USA,485.424,22.961,2020-04-12,190328.000,2.842
5,Colorado,US,04-12-2020,39.060,-105.311,7307,289,0.000,0.000,8.000,...,0.000,1376.000,0.000,84000008.000,USA,615.390,18.831,2020-04-12,34873.000,3.955
6,Connecticut,US,04-12-2020,41.598,-72.755,12035,554,0.000,0.000,9.000,...,0.000,1654.000,0.000,84000009.000,USA,1156.148,13.743,2020-04-12,41220.000,4.603
7,Delaware,US,04-12-2020,39.319,-75.507,1625,49,191.000,1385.000,10.000,...,0.000,190.000,0.000,84000010.000,USA,1140.215,11.692,2020-04-12,11103.000,3.015
8,Florida,US,04-12-2020,27.766,-81.687,19895,461,0.000,0.000,12.000,...,0.000,2772.000,0.000,84000012.000,USA,1508.520,13.933,2020-04-12,323996.000,2.317
9,Georgia,US,04-12-2020,33.041,-83.643,12452,433,0.000,0.000,13.000,...,0.000,2505.000,0.000,84000013.000,USA,537.044,20.117,2020-04-12,54453.000,3.477


In [50]:
testView.describe()

,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
count,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41800.000,41550.000,41550.000
mean,39.466,-93.668,749196.122,11167.770,30669.813,21378.317,29.320,11547.205,8670962.377,754.641,1.096,84000029.320,130538.978,1.473,312579.096,0.843
std,6.080,19.146,1177240.222,15585.131,123387.509,77075.362,15.624,9149.047,17554437.753,5235.154,0.790,15.624,127544.814,4.374,1207473.429,1.776
min,21.094,-157.498,270.000,0.000,0.000,0.000,1.000,28.743,0.000,0.000,0.000,84000001.000,0.000,0.000,0.000,0.000
25%,35.630,-105.311,99544.250,1574.000,0.000,0.000,17.000,2855.756,0.000,0.000,0.000,84000017.000,26072.185,0.000,0.000,0.000
50%,40.000,-89.648,341046.000,5598.500,0.000,0.000,29.500,10274.517,3127878.000,0.000,1.239,84000029.500,105147.639,0.000,0.000,0.000
75%,43.327,-79.806,883742.250,14018.000,4780.500,4544.250,42.000,17587.506,9842443.000,0.000,1.583,84000042.000,188195.390,0.000,16097.000,0.691
max,61.371,-69.382,10644446.000,93319.000,2470308.000,1408516.000,56.000,58927.992,318233196.000,89995.000,6.283,84000056.000,1635860.889,38.501,19565151.000,9.741
